pip install prophet --quiet - встановіть через термінал.
pip install pandas
pip install matplotlib

In [1]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt


c:\Users\natal\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


Loading of DataSet, Some checks and data prep

In [3]:
# Load dataset
df = pd.read_csv('dataset_pcs (1).csv')


In [4]:
# Show first few rows
df.head()

,period,sku_id,category_id,brand,lifetime_days,qty_total,orders_qty,abc,total_abc_numeric,avg_discount_perc_by_goods,...,war,covid,year,sin_quarter,cos_quarter,sin_month,cos_month,sin_week,cos_week,data_status
0,9/16/2024,46303774477,672241,747920273,-70,0,0,NaN,NaN,NaN,...,1,0,2024,-1,0,-1,0,"-0,9709","-0,2393",inconsistent
1,9/9/2024,46303774477,672241,747920273,-77,0,0,NaN,NaN,NaN,...,1,0,2024,-1,0,-1,0,"-0,935","-0,3546",inconsistent
2,9/16/2024,46303862950,672241,747920273,-70,0,0,NaN,NaN,NaN,...,1,0,2024,-1,0,-1,0,"-0,9709","-0,2393",inconsistent
3,9/9/2024,46303862950,672241,747920273,-77,0,0,NaN,NaN,NaN,...,1,0,2024,-1,0,-1,0,"-0,935","-0,3546",inconsistent
4,9/16/2024,46303877432,672241,747920273,-70,0,0,NaN,NaN,NaN,...,1,0,2024,-1,0,-1,0,"-0,9709","-0,2393",inconsistent


In [5]:
# Check the column names in your dataset
df.columns

Index(['period', 'sku_id', 'category_id', 'brand', 'lifetime_days',
       'qty_total', 'orders_qty', 'abc', 'total_abc_numeric',
       'avg_discount_perc_by_goods', 'max_discount_perc_by_goods',
       'Unnamed: 11', 'avg_goods_price_by_goods', 'revenue',
       'last_goods_sell_status', 'oos__by_goods', 'war', 'covid', 'year',
       'sin_quarter', 'cos_quarter', 'sin_month', 'cos_month', 'sin_week',
       'cos_week', 'data_status'],
      dtype='object')

In [6]:
#  Convert 'period' to datetime
df['period'] = pd.to_datetime(df['period'])

In [7]:
# Fix numeric columns with commas instead of dots
columns_to_fix = ['avg_discount_perc_by_goods', 'oos__by_goods', 'sin_month', 'cos_month']

for col in columns_to_fix:
    df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [8]:
# Select only required columns (adjust as needed)
df = df[['period', 'sku_id', 'qty_total', 'avg_discount_perc_by_goods', 
         'oos__by_goods', 'sin_month', 'cos_month']]


In [9]:
# Prepare results container
forecast_all = []

In [10]:
# Loop through each unique SKU
sku_list = df['sku_id'].unique()

for sku in sku_list:
    sku_df = df[df['sku_id'] == sku].copy()

    # Skip SKUs with too little data
    if len(sku_df) < 10:
        continue

    # Rename columns to match Prophet requirements
    sku_df.rename(columns={'period': 'ds', 'qty_total': 'y'}, inplace=True)

    # Drop rows with missing values in regressors
    sku_df = sku_df.dropna(subset=['avg_discount_perc_by_goods', 
                                   'oos__by_goods', 'sin_month', 'cos_month'])
 
     # Drop rows with missing regressor values
    sku_df = sku_df.dropna(subset=['avg_discount_perc_by_goods', 
                                   'oos__by_goods', 'sin_month', 'cos_month'])

    # Skip if less than 2 rows after cleaning
    if len(sku_df) < 2:
        continue
    # Initialize Prophet with additional regressors
    model = Prophet()
    model.add_regressor('avg_discount_perc_by_goods')
    model.add_regressor('oos__by_goods')
    model.add_regressor('sin_month')
    model.add_regressor('cos_month')

    model.fit(sku_df)

    # Create future dataframe for 28 days
    future = model.make_future_dataframe(periods=28)

    # Add future regressor values (using last known value)
    for col in ['avg_discount_perc_by_goods', 'oos__by_goods', 'sin_month', 'cos_month']:
        last_val = sku_df[col].iloc[-1]
        future[col] = last_val  # repeat last known value

    # Forecast
    forecast = model.predict(future)


    result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(28).copy()
    result.rename(columns={
        'ds': 'date',
        'yhat': 'forecast_qty_total',
        'yhat_lower': 'forecast_qty_total_lower',
        'yhat_upper': 'forecast_qty_total_upper'
    }, inplace=True)


    result['sku_id'] = sku


    forecast_all.append(result)

22:34:19 - cmdstanpy - INFO - Chain [1] start processing
22:34:22 - cmdstanpy - INFO - Chain [1] done processing
22:34:22 - cmdstanpy - INFO - Chain [1] start processing
22:34:23 - cmdstanpy - INFO - Chain [1] done processing
22:34:23 - cmdstanpy - INFO - Chain [1] start processing
22:34:23 - cmdstanpy - INFO - Chain [1] done processing
22:34:23 - cmdstanpy - INFO - Chain [1] start processing
22:34:23 - cmdstanpy - INFO - Chain [1] done processing
22:34:23 - cmdstanpy - INFO - Chain [1] start processing
22:34:23 - cmdstanpy - INFO - Chain [1] done processing
22:34:23 - cmdstanpy - INFO - Chain [1] start processing
22:34:23 - cmdstanpy - INFO - Chain [1] done processing
22:34:23 - cmdstanpy - INFO - Chain [1] start processing
22:34:24 - cmdstanpy - INFO - Chain [1] done processing
22:34:24 - cmdstanpy - INFO - Chain [1] start processing
22:34:24 - cmdstanpy - INFO - Chain [1] done processing
22:34:24 - cmdstanpy - INFO - Chain [1] start processing
22:34:24 - cmdstanpy - INFO - Chain [1]

Getting a result, saving to Csv.

In [11]:
#  Combine all forecasts
final_forecast_df = pd.concat(forecast_all, ignore_index=True)


In [12]:

# Save to CSV
final_forecast_df.to_csv('forecast_28_days_all_skus_with_regressors.csv', index=False)

In [13]:

# Show sample of result
final_forecast_df.head()

,date,forecast_qty_total,forecast_qty_total_lower,forecast_qty_total_upper,sku_id
0,2024-12-24,1.0,1.0,1.0,46303774477
1,2024-12-25,1.0,1.0,1.0,46303774477
2,2024-12-26,1.0,1.0,1.0,46303774477
3,2024-12-27,1.0,1.0,1.0,46303774477
4,2024-12-28,1.0,1.0,1.0,46303774477
